### --- Day 12: Passage Pathing ---
With your submarine's subterranean subsystems subsisting suboptimally, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just a path - the only way to know if you've found the best path is to find all of them.

Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:

    start-A
    start-b
    A-c
    A-b
    b-d
    A-end
    b-end
This is a list of how all of the caves are connected. You start in the cave named start, and your destination is the cave named end. An entry like b-d means that cave b is connected to cave d - that is, you can move between them.

So, the above cave system looks roughly like this:

        start
        /   \
    c--A-----b--d
        \   /
         end
Your goal is to find the number of distinct paths that start at start, end at end, and don't visit small caves more than once. There are two types of caves: big caves (written in uppercase, like A) and small caves (written in lowercase, like b). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should visit small caves at most once, and can visit big caves any number of times.

Given these rules, there are 10 paths through this example cave system:

    start,A,b,A,c,A,end
    start,A,b,A,end
    start,A,b,end
    start,A,c,A,b,A,end
    start,A,c,A,b,end
    start,A,c,A,end
    start,A,end
    start,b,A,c,A,end
    start,b,A,end
    start,b,end
(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)

Note that in this cave system, cave d is never visited by any path: to do so, cave b would need to be visited twice (once on the way to cave d and a second time when returning from cave d), and since cave b is small, this is not allowed.

Here is a slightly larger example:

    dc-end
    HN-start
    start-kj
    dc-start
    dc-HN
    LN-dc
    HN-end
    kj-sa
    kj-HN
    kj-dc
The 19 paths through it are as follows:

    start,HN,dc,HN,end
    start,HN,dc,HN,kj,HN,end
    start,HN,dc,end
    start,HN,dc,kj,HN,end
    start,HN,end
    start,HN,kj,HN,dc,HN,end
    start,HN,kj,HN,dc,end
    start,HN,kj,HN,end
    start,HN,kj,dc,HN,end
    start,HN,kj,dc,end
    start,dc,HN,end
    start,dc,HN,kj,HN,end
    start,dc,end
    start,dc,kj,HN,end
    start,kj,HN,dc,HN,end
    start,kj,HN,dc,end
    start,kj,HN,end
    start,kj,dc,HN,end
    start,kj,dc,end
Finally, this even larger example has 226 paths through it:

    fs-end
    he-DX
    fs-he
    start-DX
    pj-DX
    end-zg
    zg-sl
    zg-pj
    pj-he
    RW-he
    fs-DX
    pj-RW
    zg-RW
    start-pj
    he-WI
    zg-he
    pj-fs
    start-RW
How many paths through this cave system are there that visit small caves at most once?

In [347]:
def paths_input(input_file:str):
    """Build dict from path text file."""
    paths = []
    with open(input_file, 'r') as file:
        for i, line in enumerate(file):
            nodes = line.strip().split('-')
            paths.append({'id':i, 'start':nodes[0], 'end':nodes[1], 'directionality':'normal'})            
    return paths



def all_available_paths(path_list):
    """ 
    Now modify dict so it has both directions for all paths that way only have to worry about single
    key value to check later on, with the exception of proper start or end paths (these can only go in one direction)
    """
    all_directions = []
    for p in path_list:
        path = dict(p)
        if (
            path['start'] != 'start' 
            and path['start'] != 'end' 
            and path['end'] != 'start'
            and path['end'] != 'end'
        ):
            all_directions.append(dict(path))
            path['end'], path['start'], path['directionality'] = path['start'], path['end'], 'reversed'
            all_directions.append(path)
        # Reverse path for the end nodes (currently 'end' is at 'start' key)
        elif path['start'] == 'end' or path['end'] == 'start':
            path['end'], path['start'], path['directionality'] = path['start'], path['end'], 'reversed'
            all_directions.append(path)
        else:
            all_directions.append(path)
    return all_directions


In [352]:
def find_route(available_paths:list, current_path:list, found_routes:list, small_cave_visits=1, small_cave_revisited=False) -> list:
    """
    Recurse through available routes.  Build list of current_path, adding to it if conditions met. 
    found_routes is built onto and eventually output.
    """
    for path in available_paths:
        
        # Small cave switch
        small_cave = True if path['end'] == path['end'].lower() else False
         
        # First node
        if len(current_path) == 0:
            
            # Nested if so don't pass on empty list
            if path['start'] == 'start':
                new_current_path = current_path.copy()
                new_current_path.append(path['start'])
                new_current_path.append(path['end'])
                
                # Recurse
                find_route(available_paths, new_current_path, found_routes, small_cave_visits, small_cave_revisited)

        # Is this the final path?
        elif (
            path['start'] == current_path[-1] 
            and path['end'] == 'end' 
        ):
            # Append new path
            new_current_path = current_path.copy()
            new_current_path.append(path['end'])
            found_routes.append(new_current_path)
        
        # Not final but can be joined?
        # Path matches up with last
        elif (
            path['start'] == current_path[-1]
            and (
                small_cave == False 
                or (
                    small_cave == True 
                    # Check if small cave it's not been revisited or is well below 
                    and (
                        current_path.count(path['end']) < (small_cave_visits) 
                        and small_cave_revisited == False
                    )
                    or (current_path.count(path['end']) == 0)
                )
            )
        ):    
            # Append new path
            new_current_path = current_path.copy()
            new_current_path.append(path['end'])
            
            # Change small cave revisited status
            if small_cave == True and new_current_path.count(path['end']) >= small_cave_visits:
                new_small_cave_revisited = True
            else:
                new_small_cave_revisited = small_cave_revisited
                  
            # Recurse
            find_route(available_paths, new_current_path, found_routes, small_cave_visits, new_small_cave_revisited)
        
        # Path doesn't connect to current route          
        else:
            pass
        
    return found_routes

In [353]:
# Pt1 Workflow

# Input
paths = paths_input('data/d12_paths.txt')
all_directions = all_available_paths(paths)

# Find all routes - no small cave revisits.
found_routes = []
current_path = []
total_routes_pt1 = find_route(all_directions, found_routes, current_path, small_cave_visits=1)

print(f'The total number of routes available in Pt1 is: {len(total_routes_pt1)}')



# Find all routes - a single small cave can be revisited twice
found_routes = []
current_path = []
total_routes_pt2 = find_route(all_directions, found_routes, current_path, small_cave_visits=2)


print(f'The total number of routes available in Pt2 is: {len(total_routes_pt2)}')


The total number of routes available in Pt1 is: 3421
The total number of routes available in Pt2 is: 84870


In [344]:
total_routes_pt1

[['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'RL', 'wz', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'RL', 'wz', 'dm', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'RL', 'gy', 'wz', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'RL', 'gy', 'wz', 'dm', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'RL', 'gy', 'RL', 'wz', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'RL', 'gy', 'RL', 'wz', 'dm', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'wz', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'wz', 'dm', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'AV', 'gy', 'wz', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'AV', 'gy', 'wz', 'dm', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'AV', 'gy', 'RL', 'wz', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'AV', 'gy', 'RL', 'wz', 'dm', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'gy', 'wz', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'gy', 'wz', 'dm', 'end'],
 ['start', 'cg', 'AV', 'mh', 'AV', 'rf', 'gy